In [1]:
from bs4 import BeautifulSoup 
from splinter import Browser
from selenium import webdriver
import os
import pandas as pd
import time

In [2]:
# Set Executable Path & Initialize Chrome Browser
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
# Visit the NASA Mars News Site
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
#parse everything

html = browser.html
nasa_news = BeautifulSoup(html, "html.parser")
slide_name_for_title = nasa_news.select_one("ul.item_list li.slide")

In [5]:
slide_name_for_title.find("div", class_="content_title")

<div class="content_title"><a href="/news/8508/nasa-invites-students-to-name-mars-2020-rover/" target="_self">NASA Invites Students to Name Mars 2020 Rover</a></div>

In [8]:
# Scrape the Latest News Title
# Use Parent Element to Find First <a> Tag and Save it as news_title
news_title = slide_name_for_title.find("div", class_="content_title").get_text()
print(news_title)


NASA Invites Students to Name Mars 2020 Rover


In [9]:
news_p = slide_name_for_title.find("div", class_="article_teaser_body").get_text()
print(news_p)

Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


JPL Mars Space Images - Featured Image

In [10]:
# Visit the NASA JPL (Jet Propulsion Laboratory) Site
# executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
# browser = Browser("chrome", **executable_path)
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [11]:
# Ask Splinter to Go to Site and Click Button with Class Name full_image
# <button class="full_image">Full Image</button>
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [12]:
# Find "More Info" Button and Click It
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

In [13]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image = BeautifulSoup(html, "html.parser")

In [14]:
featured_image_url =image.select_one("figure.lede a img").get("src") 
print (featured_image_url)

/spaceimages/images/largesize/PIA14712_hires.jpg


In [15]:
# Use Base URL to Create Absolute URL (changing from jpg file) to hyperlinked and active
featured_image_url = f"https://www.jpl.nasa.gov{featured_image_url}"
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14712_hires.jpg


Mars Weather

In [16]:
# Visit the Mars Weather Twitter Account
# executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
# browser = Browser("chrome", **executable_path, headless=False)
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)


In [17]:
# Parse Results HTML with BeautifulSoup
html = browser.html
weather_soup = BeautifulSoup(html, "html.parser")

In [18]:
# Finding a Tweet with the data-name `Mars Weather`
mars_weather_tweet = weather_soup.find("div", 
                                       attrs={
                                           "class": "tweet", 
                                            "data-name": "Mars Weather"
                                        })


In [19]:
# Search Within Tweet for <p> Tag Containing Tweet Text
mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()
print(mars_weather)

We won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/ …pic.twitter.com/fWruE2v151


Mars Facts

In [24]:
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
print(mars_df)
mars_df.columns=["Description","Value",""]
mars_df.set_index("Description", inplace=True)
mars_df = pd.DataFrame(mars_df['Value'])
mars_df

  Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:    -153 to 20 °C      -88 to 58°C


,Value
Description,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-153 to 20 °C


Mars Hemispheres

In [25]:
# Visit the USGS Astrogeology Science Center Site
# executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
# browser = Browser("chrome", **executable_path, headless=False)
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [26]:
hemisphere_image = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    # Find Element on Each Loop to Avoid a Stale Element Exception
    browser.find_by_css("a.product-item h3")[item].click()
    
    # Find Sample Image Anchor Tag & Extract <href>
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    
    # Get Hemisphere Title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append Hemisphere Object to List
    hemisphere_image.append(hemisphere)
    
    # Navigate Backwards
    browser.back()

In [27]:
hemisphere_image

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]